In [3]:
import pandas as pd
import streamlit as st

uploaded_file = st.file_uploader("Choose a CSV file")

if uploaded_file is not None:
    df_uploaded = pd.read_csv(uploaded_file, encoding='utf-8')  # or another encoding if needed
    st.write(df_uploaded)
else:
    st.warning("Please upload a file.")

2025-04-14 19:51:10.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:51:10.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
print(uploaded_file)


None


In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
import os

# --- Theme ---
PRIMARY_COLOR = "#0e4e4e"
SECONDARY_COLOR = "#e1ebae"
BACKGROUND_COLOR = "#FFF2DF"
ACCENT_COLOR = "#80bce6"

def set_skanem_theme():
    st.markdown(f"""
        <style>
            .stApp {{ background-color: {BACKGROUND_COLOR}; }}
            h1, h2, h3, h4, h5, h6 {{ color: {PRIMARY_COLOR}; }}
            .stButton>button {{
                background-color: {PRIMARY_COLOR}; color: white;
                border-radius: 4px; border: none;
            }}
            .stButton>button:hover {{
                background-color: #0a3a3a; color: white;
            }}
            .stMetric {{ background-color: {SECONDARY_COLOR}; border-radius: 4px; padding: 10px; }}
        </style>
    """, unsafe_allow_html=True)

set_skanem_theme()

# --- Logo & Title ---
st.image("C:/Users/chris.mutuku/OneDrive - Skanem AS/Desktop/logo.jpg", width=300)
st.title("📦 Advanced Supply Chain Forecasting")

# --- File Upload ---
uploaded_file = st.file_uploader(
    "📤 Upload SKU Consumption Data (CSV or Excel)",
    type=["csv", "xlsx"],
    help="Upload file with columns: SKU, Date, Quantity"
)

def read_data(file):
    ext = os.path.splitext(file.name)[1].lower()
    if ext == ".csv":
        try:
            return pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            return pd.read_csv(file, encoding='ISO-8859-1')
    else:
        return pd.read_excel(file)

# --- Main Logic ---
if uploaded_file:
    try:
        df = read_data(uploaded_file)
        if 'Date' not in df.columns:
            st.warning("⚠️ No 'Date' column found in the uploaded data")
        else:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values('Date')

            required_cols = ['SKU', 'Date', 'Quantity']
            if not all(col in df.columns for col in required_cols):
                st.warning("⚠️ Please ensure your data contains these columns: SKU, Date, Quantity")
            else:
                st.success("✅ Data loaded successfully")

                with st.expander("📁 View Raw Data"):
                    st.dataframe(df)

                sku_list = df['SKU'].unique().tolist()
                selected_sku = st.selectbox("🔍 Select SKU to analyze", sku_list)
                sku_data = df[df['SKU'] == selected_sku]

                monthly = sku_data.set_index('Date').resample('M')['Quantity'].sum().reset_index()
                monthly['Month'] = monthly['Date'].dt.to_period('M')

                col1, col2, col3 = st.columns(3)
                col1.metric("Total", f"{sku_data['Quantity'].sum():,}")
                col2.metric("Avg Monthly", f"{monthly['Quantity'].mean():,.1f}")
                col3.metric("Last", f"{sku_data.iloc[-1]['Quantity']:,}",
                            delta=sku_data['Date'].max().strftime('%b %Y'))

                st.subheader(f"📈 Trend for {selected_sku}")
                fig = px.line(monthly, x='Date', y='Quantity',
                              markers=True,
                              color_discrete_sequence=[PRIMARY_COLOR],
                              labels={'Quantity': 'Consumption', 'Date': 'Month'})
                fig.update_layout(plot_bgcolor=BACKGROUND_COLOR,
                                  paper_bgcolor=BACKGROUND_COLOR,
                                  hovermode="x unified")
                st.plotly_chart(fig, use_container_width=True)

                # Forecasting
                st.subheader("🔮 Forecasting")
                col1, col2 = st.columns(2)
                months = col1.number_input("Forecast Months", 1, 24, 6)
                model_type = col2.selectbox("Model", ["Exponential Smoothing", "Prophet"])

                if st.button("Generate Forecast"):
                    with st.spinner("Forecasting in progress..."):
                        try:
                            ts = monthly.set_index('Date')['Quantity']
                            if model_type == "Exponential Smoothing":
                                model = ExponentialSmoothing(ts, trend='add', seasonal='add', seasonal_periods=12).fit()
                                forecast = model.forecast(months)
                            else:
                                prophet_df = monthly[['Date', 'Quantity']].rename(columns={'Date': 'ds', 'Quantity': 'y'})
                                model = Prophet(yearly_seasonality=True)
                                model.fit(prophet_df)
                                future = model.make_future_dataframe(periods=months, freq='M')
                                forecast_df = model.predict(future)
                                forecast = forecast_df['yhat'][-months:].values

                            # Fixed Forecast Start Date Logic
                            last_date = monthly['Date'].iloc[-1]
                            forecast_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=months, freq='M')

                            forecast_df = pd.DataFrame({'Date': forecast_dates, 'Quantity': forecast, 'Type': 'Forecast'})
                            hist_df = pd.DataFrame({'Date': monthly['Date'], 'Quantity': monthly['Quantity'], 'Type': 'Historical'})
                            combined = pd.concat([hist_df, forecast_df])

                            chart = go.Figure()
                            chart.add_trace(go.Scatter(x=hist_df['Date'], y=hist_df['Quantity'], name="Historical",
                                                       mode='lines+markers', line=dict(color=PRIMARY_COLOR)))
                            chart.add_trace(go.Scatter(x=forecast_df['Date'], y=forecast_df['Quantity'], name="Forecast",
                                                       mode='lines+markers', line=dict(color=ACCENT_COLOR, dash='dot')))
                            chart.update_layout(title=f"{months}-Month Forecast for {selected_sku}",
                                                plot_bgcolor=BACKGROUND_COLOR, paper_bgcolor=BACKGROUND_COLOR,
                                                hovermode="x unified", xaxis_title="Month", yaxis_title="Quantity")
                            st.plotly_chart(chart, use_container_width=True)

                            # Forecast Table + Download
                            st.subheader("📋 Forecast Table")
                            forecast_table = forecast_df.copy()
                            forecast_table['Date'] = forecast_table['Date'].dt.strftime('%Y-%m')
                            forecast_table = forecast_table.rename(columns={'Date': 'Forecast Month', 'Quantity': 'Forecasted Quantity'})
                            st.dataframe(forecast_table)

                            csv = forecast_table.to_csv(index=False).encode('utf-8')
                            st.download_button("💾 Download Forecast CSV", csv, f"{selected_sku}_forecast.csv", "text/csv")

                        except Exception as e:
                            st.error(f"❌ Error generating forecast: {e}")

    except Exception as e:
        st.error(f"❌ Error reading file: {e}")
else:
    st.info("ℹ️ Please upload a CSV or Excel file to begin SKU analysis.")

# --- Sample Template ---
with st.expander("💡 No file? Download sample template"):
    sample = pd.DataFrame({'SKU': ['SKU001', 'SKU001'], 'Date': ['2023-01-01', '2023-02-01'], 'Quantity': [100, 120]})
    csv = sample.to_csv(index=False).encode('utf-8')
    st.download_button("⬇️ Download Sample CSV", csv, "sample_sku.csv", "text/csv")


2025-04-14 19:45:40.179 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.646 
  command:

    streamlit run c:\Users\chris.mutuku\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-14 19:45:40.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 19:45:40.683 Th